# Import Libraries

In [167]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
%matplotlib inline


#import bs4 as bs or:
from bs4 import BeautifulSoup
#BeautifulSoup is a Python library for pulling data out of HTML and XML files

import re
import requests
import urllib.request as url
#rllib.request is a Python module for fetching URLs (Uniform Resource Locators).It offers a simple
#interface, in the form of the urlopen function.

from time import sleep
#Need to use a delay between page scrapes in order to limit getting blocked by Yelp

# Step One: 

In [168]:
# Make a get request to retrieve the page
html_page = requests.get('https://www.yelp.com/search?find_desc=Clothing%20Boutiques&find_loc=San%20Francisco%2C%20CA&ns=1&cflt=womenscloth')

# Pass the page contents to beautiful soup for parsing
soup = BeautifulSoup(html_page.content, 'html.parser')

In [169]:
#to iterate over all pages and stores:
for i in range(1,416):
    #find a URL pattern for different pages ,total num_stores = 416
    url="https://www.yelp.com/search?find_desc=Clothing%20Boutiques&find_loc=San%20Francisco%2C%20CA&ns=1&cflt=womenscloth&start={}".format((i//40)*40)
    search_page=url
    #print(search_page)

### Set main and secondary attributes and class names to each of required features:

In [170]:
main_attributes_class="lemon--div__373c0__1mboc mainAttributes__373c0__1r0QA arrange-unit__373c0__o3tjT arrange-unit-fill__373c0__3Sfw1 border-color--default__373c0__3-ifU"
#including:
business_name_class="lemon--div__373c0__1mboc businessName__373c0__1fTgn display--inline-block__373c0__1ZKqC border-color--default__373c0__3-ifU"
rating_class="lemon--span__373c0__3997G display--inline__373c0__3JqBP border-color--default__373c0__3-ifU"
price_range_class="lemon--span__373c0__3997G text__373c0__2Kxyz priceRange__373c0__2DY87 text-color--black-extra-light__373c0__2OyzO text-align--left__373c0__2XGa- text-bullet--after__373c0__3fS1Z"
review_count_class="lemon--span__373c0__3997G text__373c0__2Kxyz reviewCount__373c0__2r4xT text-color--black-extra-light__373c0__2OyzO text-align--left__373c0__2XGa-"

secondary_attributes_class="lemon--div__373c0__1mboc secondaryAttributes__373c0__7bA0w arrange-unit__373c0__o3tjT border-color--default__373c0__3-ifU"
#inclufing:
business_phonenumber_class="lemon--div__373c0__1mboc display--inline-block__373c0__1ZKqC border-color--default__373c0__3-ifU"
business_address_class="lemon--span__373c0__3997G raw__373c0__3rcx7"
# business_district_class="lemon--p__373c0__3Qnnj text__373c0__2Kxyz text-color--black-extra-light__373c0__2OyzO text-align--right__373c0__1f0KI text-size--small__373c0__3NVWO"

In [171]:
#find all the div classes/main_attributes which refers to all the stores listed on the search result page and set them to mains
mains=soup.find_all("div",{"class":main_attributes_class})

#find all the div classes/secondary_attributes which refers to all the stores listed on the search result page and set them to mains and secondary_attributes 
secondaries=soup.find_all("div",{"class":secondary_attributes_class})

In [172]:
boutique_name=[]
ratings=[]
number_reviews=[]
price_range=[]
boutique_url_address=[]

for main in mains:
    try:
        boutique_name.append(main.find("a").text)
    except:
        boutique_name.append(None)
    try:
        ratings.append(main.find("span",{"class":rating_class}).div.get("aria-label"))
    except:
        ratings.append(None)
    try:
        number_reviews.append(main.find("span",{"class":review_count_class}).text)
    except:
        number_reviews.append(None)
    try:
        price_range.append(main.find("span",{"class":price_range_class}).get_text())
    except:
        price_range.append(None)
    try:
        base_url="https://www.yelp.com"
        business_name_url=main.find('a').attrs['href']
        boutique_url_address.append(base_url+business_name_url)
    except:
        boutique_url_address.append(None)

In [173]:
phone_number=[]
address=[]

for secondary in secondaries:
    try:
        phone_number.append(secondary.find("p").text)
    except:
        phone_number.append(None)
    try:
        address.append(secondary.find("span",{"class":business_address_class}).get_text())
    except:
        address.append(None) 

In [174]:
data={"boutique_name":boutique_name,"ratings":ratings,"number_reviews":number_reviews,
      "price_range":price_range,"phone_number":phone_number,"address":address,"boutique_url_address":boutique_url_address}
boutiques_info=pd.DataFrame(data)
coulmns=["boutique_name","ratings","number_reviews","price_range","phone_number","address","boutique_url_address"]
boutiques_info.head()
#boutiques_info.to_csv("boutiques_info",columns=columns)

,boutique_name,ratings,number_reviews,price_range,phone_number,address,boutique_url_address


# step 2:
Define a function to return list of reviews for a business across all of the review pages. For each page, it will call the  review scraper (see below) to gather all review data for the page:

In [175]:
#retrieve reviews for each boutique:
#each page has 20 reviews
def retrieve_reviews(boutique_name, boutique_url_address, number_reviews):
    
    url = boutique_url_address
    total_reviews=[]
        
    for i in range(0, int(number_reviews)//20 + 1):
        url_page = url + "&start={}".format(i*20)
        html_page = requests.get(url_page)
        soup = BeautifulSoup(html_page.content, 'html.parser')
        reviews = soup.find_all("span",{"class":"lemon--span__373c0__3997G raw__373c0__3rKqk"})
        
        for review in reviews:
            total_reviews.append(review.text)
        
    return total_reviews

In [176]:
#check the results for one store:
boutique_name = 'Kisha Studio Fashion Boutique'
boutique_url_address = 'https://www.yelp.com/biz/kisha-studio-fashion-boutique-san-francisco-2?osq=Clothing+Boutiques'
number_reviews = 119
myreviews = retrieve_reviews(boutique_name, boutique_url_address, number_reviews)

for review in myreviews:
    print(review,"\n")

print(len(myreviews))

0


# step 3:

Writing a functions that will extract a list of each of required features for **each web page**.

In [177]:
def retrieve_boutique_names(soup):
    mains=soup.find_all("div",{"class":main_attributes_class})
    boutique_name=[]
    for main in mains:
        try:
            boutique_name.append(main.find("a").text)
        except:
            boutique_name.append(None)
    return boutique_name

In [178]:
def retrieve_ratings(soup):
    mains=soup.find_all("div",{"class":main_attributes_class})
    ratings=[]
    for main in mains:
        try:
            ratings.append(main.find("span",{"class":rating_class}).div.get("aria-label"))
        except:
            ratings.append(None)
    return ratings

In [179]:
def retrieve_number_reviews(soup):
    mains=soup.find_all("div",{"class":main_attributes_class})
    number_reviews=[]
    for main in mains:
        try:
            number_reviews.append(main.find("span",{"class":review_count_class}).text)
        except:
            number_reviews.append(None)
    return number_reviews

In [180]:
def retrieve_price_range(soup):
    mains=soup.find_all("div",{"class":main_attributes_class})
    price_range=[]
    for main in mains:
        try:
            price_range.append(main.find("span",{"class":price_range_class}).get_text())
        except:
            price_range.append(None)
    return price_range

In [181]:
def retrieve_url_address(soup):
    mains=soup.find_all("div",{"class":main_attributes_class})
    url_address=[]
    for main in mains:
        try:
            base_url="https://www.yelp.com"
            business_name_url=main.find('a').attrs['href']
            url_address.append(base_url+business_name_url)
        except:
            url_address.append(None) 
    return url_address

In [182]:
def retrieve_phone_number(soup):
    secondaries=soup.find_all("div",{"class":secondary_attributes_class})
    phone_number=[]
    for secondary in secondaries:
        try:
            phone_number.append(secondary.find("p").text)
        except:
            phone_number.append(None)            
    return phone_number

In [183]:
def retrieve_address(soup):
    secondaries=soup.find_all("div",{"class":secondary_attributes_class})
    address=[]
    for secondary in secondaries:
        try:
            address.append(secondary.find("span",{"class":business_address_class}).get_text())
        except:
            address.append(None)             
    return address

In [184]:
# def retrieve_reviews(soup):
#     reviews=soup.find_all("p",{"class":"lemon--p__373c0__3Qnnj text__373c0__2Kxyz text-color--black-extra-light__373c0__2OyzO text-align--left__373c0__2XGa-"})
#     total_reviews=[]
#     for review in reviews:
#         total_reviews.append(review.text)
#     return total_reviews

# Step 4:

Define a function to combine all the business information from all search pages for womens clothing boutiques in san francisco

In [192]:
def retrieve_business_info(num_stores):
    boutique_names=[]
    ratings=[]
    number_reviews=[]
    price_range=[]
    url_address=[]
    phone_number=[]
    address=[]

    #total num_stores in all pages = 416
    #num_stores = 60
    for i in range(0, num_stores, 40):
        print(i)
        #print("search_pages:",i)
        #find a url pattern for all search pages
        url="https://www.yelp.com/search?find_desc=Clothing%20Boutiques&find_loc=San%20Francisco%2C%20CA&ns=1&cflt=womenscloth&start={}".format(i)
        #Make a get request to retrieve each page
        html_page = requests.get(url)
        #Pass the page contents to beautiful soup for parsing
        soup = BeautifulSoup(html_page.content, 'html.parser')
        boutique_names += retrieve_boutique_names(soup)
        #print('1: ', len(boutique_names))
        ratings += retrieve_ratings(soup)
        #print('2: ', len(ratings))
        price_range += retrieve_price_range(soup)
        #print('3: ', len(price_range))
        url_address += retrieve_url_address(soup)
        #print('4: ', len(url_address))
        phone_number += retrieve_phone_number(soup)
        #print('5: ', len(phone_number))
        address += retrieve_address(soup)
        #print('6: ', len(address))
        number_reviews += retrieve_number_reviews(soup)
        #print('7: ', len(number_reviews))        
                
        #Random delay between 1 and 4 seconds to prevent getting blocked
        sleep(np.random.randint(1,3))
    
    data={"boutique_names":boutique_names,"ratings":ratings,"number_reviews":number_reviews,
      "price_range":price_range,"phone_number":phone_number,"address":address,"url_address":url_address}
    df=pd.DataFrame(data)

    #df=pd.DataFrame([boutique_names,ratings,price_range,phone_number,address,url_address])#.transpose()
    #df.columns=["boutique_name","ratings","price_range","phone_number","address","url_address"]

    return df

# print(len(df))
# df.head()

# Step 5:
Saving the dataframe into a CSV file:

In [193]:
#save the dataframe as a csv file:
business_info_df = retrieve_business_info(num_stores=416)
business_info_df.to_csv("sf_wclothing_boutiques.csv")

0
40
80
120
160
200
240
280
320
360
400


# Step 6:
Define a function to loop through all the boutiques in business_info_df and get all the reviews for each:

In [197]:
def boutique_reviews(num_business): 

    #set container
    last_index_completed = 1
    boutique_reviews = []
    total_data = []

    for last_index_completed in range(1, num_business):
        biz = business_info_df.iloc[last_index_completed]

        #Random delay between 1 and 5 seconds to try to prevent getting blocked
        sleep(np.random.randint(2,5))
        try:
            print(last_index_completed,":", int(biz['number_reviews']),"reviews\t", biz['boutique_names'], biz['url_address'], end='')
            #Get all reviews for the business and add to list.
            boutique_reviews = retrieve_reviews(biz['boutique_names'],biz['url_address'],biz['number_reviews'])
            print(" completed")
        except:
            print(last_index_completed,":","missing_info")
        
        data={"boutique_names":biz['boutique_names'],"number_reviews":biz['number_reviews'],
              "url_address":biz['url_address'],"reviews":boutique_reviews}
        total_data.append(data)
    
    df=pd.DataFrame(total_data)        
    #df = pd.DataFrame(boutique_reviews)
    return df 

In [199]:
boutique_reviews_df = boutique_reviews(num_business=45)
boutique_reviews_df.to_csv("sf_wclothing_boutiques_reviews.csv")
#len(boutique_reviews_df)

1 : 119 reviews	 Kisha Studio Fashion Boutique https://www.yelp.com/biz/kisha-studio-fashion-boutique-san-francisco-2?osq=Clothing+Boutiques completed
2 : 29 reviews	 Isalis https://www.yelp.com/biz/isalis-san-francisco?osq=Clothing+Boutiques completed
3 : 17 reviews	 Onyx https://www.yelp.com/biz/onyx-san-francisco-3?osq=Clothing+Boutiques completed
4 : 26 reviews	 Asmbly Hall https://www.yelp.com/biz/asmbly-hall-san-francisco?osq=Clothing+Boutiques completed
5 : 24 reviews	 Two Birds https://www.yelp.com/biz/two-birds-san-francisco-2?osq=Clothing+Boutiques completed
6 : 400 reviews	 Therapy Stores https://www.yelp.com/biz/therapy-stores-san-francisco?osq=Clothing+Boutiques completed
7 : 29 reviews	 Wild Feather https://www.yelp.com/biz/wild-feather-san-francisco?osq=Clothing+Boutiques completed
8 : 32 reviews	 Gravel & Gold https://www.yelp.com/biz/gravel-and-gold-san-francisco?osq=Clothing+Boutiques completed
9 : 23 reviews	 Current Clothing https://www.yelp.com/biz/current-clothing

In [220]:
# for main in mains:
#     try:
#         storename=main.find("a").text
#         print("Boutique_name:"+storename)
#     except:
#         print(None)
#     try:
#         ratings=main.find("span",{"class":rating_class}).div.get("aria-label")
#         print("Boutique_rating:"+ratings)
#     except:
#         print(None)
#     try:
#         noreviews=main.find("span",{"class":review_count_class}).text
#         print("Boutique_reviews:"+noreviews)
#     except:
#         print(None)
#     try:
#         price= main.find("span",{"class":price_range_class}).get_text()
#         print("Boutique_pricerange:"+price)
#     except:
#         print(None)
#     try:
#         base_url="https://www.yelp.com"
#         business_name_url=main.find('a').attrs['href']
#         print("Boutique_Website:"+base_url+business_name_url)
#     except:
#         print(None)

In [191]:
# for secondary in secondaries:
#     try:
#         phone_num=secondary.find("p").text
#         print("Boutique_phonenumber:"+phone_num)
#     except:
#         print(None)
#     try:
#         address=secondary.find("span",{"class":business_address_class}).get_text()
#         print("Boutique_address:"+address)
#     except:
#         print(None) 
# #how to retrieve district?

In [219]:
# for main in mains:   
#     try:
#         base_url="https://www.yelp.com"
#         business_name_url=main.find('a').attrs['href']
#         print("Boutique_Website:"+base_url+business_name_url)
#     except:
#         print(None)

In [209]:
# reviews=soup.find_all("p",{"class":"lemon--p__373c0__3Qnnj text__373c0__2Kxyz text-color--black-extra-light__373c0__2OyzO text-align--left__373c0__2XGa-"})
# for review in reviews:
#     print(review.text,"\n")

In [107]:
# check the results for one store:
# html_page = requests.get("https://www.yelp.com/biz/kisha-studio-fashion-boutique-san-francisco-2?osq=Clothing+Boutiques&start=0")
# soup = BeautifulSoup(html_page.content, 'html.parser')
# reviews = soup.find_all("span",{"class":"lemon--span__373c0__3997G raw__373c0__3rKqk"})
# for review in reviews:
#     print(review.text,"\n")